# Pipeline job using data transfer to copy data

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `datatranster` component using load component function.
- Create `PipelineJob` with `datatranster` component.

**Motivations** - This notebook covers the scenario that user define pipeline component using dsl decorator.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, command, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

In [ ]:
from azure.ai.ml._version import VERSION

print(VERSION)

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Define and create components into workspace
## 2.1 Load data transfer components from YAML

In [ ]:
components_dir = "/../../../../../../cli/assets/component"
data_merge = load_component(source=f"{components_dir}/data_merge.yml")

## 2.2 (Optional) Create data transfer component to workspace

In [ ]:
registered_data_pipeline_component = ml_client.components.create_or_update(
    data_merge
)

## 2.3 Define command component via command function
Use `command` function to create a `Command` object which can be used in `@pipeline` function.

In [ ]:
environment = "AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1"
# Create a dummy command component to generate output
hello_step = command(
    name="hello_step",
    display_name="hello step",
    description="generate dummy folder as output",
    tags=dict(),
    command="echo 'hello' > ${{outputs.hello_output}}/hello.txt",
    environment=environment,
    outputs=dict(
        hello_output=Output(type="uri_folder")
    ),
)

world_step = command(
    name="world_step",
    display_name="world_step step",
    description="generate dummy folder as output",
    tags=dict(),
    command="echo 'world' > ${{outputs.world_output}}/world.txt",
    environment=environment,
    outputs=dict(
        hello_output=Output(type="uri_folder")
    ),
)

# 3. Build pipeline job to merge data
## 3.1 Build pipeline

In [ ]:
# Construct pipeline
@pipeline
def pipleine_copy_data():
    # 
    data_pipeline_node = data_pipeline(
        raw_data=pipeline_raw_data,
    )
    # Use train pipeline component
    train_pipeline_node = train_pipeline(
        train_data=data_pipeline_node.outputs.train_data,
        compute_train_node=compute_train_node,
    )
    # Return: pipeline outputs
    return {
        "pipeline_job_trained_model": train_pipeline_node.outputs.trained_model,
        "pipeline_job_predictions": train_pipeline_node.outputs.predictions,
        "pipeline_job_score_report": train_pipeline_node.outputs.score_report,
    }


pipeline_job = nyc_taxi_data_regression(
    pipeline_raw_data=Input(type="uri_folder", path="./data/"),
    compute_train_node="gpu-cluster",
)

# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"

In [ ]:
# Inspect built pipeline
print(pipeline_job)

## 3.2 Submit pipeline job

In [ ]:
# Submit pipeline job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="nyc_taxi_data_regression_with_pipeline_component"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../README.md)